In [9]:
from data import load_data, SOURCES, save_to_json, load_from_json
from models import (
    get_gpt_recognition_logprobs,
    get_model_choice,
    get_logprobs_choice_with_sources,
    get_gpt_score,
    get_gpt_summary_similarity,
    get_gpt_paraphrase,
    get_gpt_choice_null_baseline
)

from math import exp
from pprint import pprint
from random import shuffle

from tqdm import tqdm
import time
import os

import random
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

import re
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np



In [ ]:
def generate_gpt_detect_null(
    dataset,
    model,
    comparison_models,
    shuffle=True,
    starting_idx=500,
    ending_idx = 1000
):
    exact_model = model

    responses, articles, keys = load_data(dataset)

    results = []  

    for key in tqdm(keys[starting_idx:ending_idx], desc="Processing keys"):
        article = articles[key]
        first_summary = responses[comparison_models[0]][key]
        second_summary = responses[comparison_models[1]][key]
        # Shuffle at every key
        if shuffle:
            summaries = [first_summary, second_summary]
            random.shuffle(summaries)
            first_summary, second_summary = summaries
        
        result = {"key": key}

        try:
            forward_result = get_gpt_choice_null_baseline(
                first_summary,
                second_summary,
                article,
                return_logprobs=True,
            )

            if shuffle:
                logprob_for_3 = next((item.logprob for item in forward_result if item.token == '3'), None) # item.token 1 or 2 for GT, 3 for null.
            else:
                logprob_for_3 = next((item.logprob for item in forward_result if item.token == '1'), None) # item.token 1 or 2 for GT, 3 for null.
            result["detection_score"] = exp(logprob_for_3)

        except ValueError:
            print('Error:', key)
        
        results.append(result)
    return results



In [14]:
for model in ["gpt4"]:
    
    print(f"Starting {model}")
    comparison_models =  ["gpt4", "gpt35"] ## Change for the two models
    print(comparison_models)
    #shuffle=False for GT
    results = generate_gpt_detect_null(
        "cnn", model,comparison_models, shuffle=False, starting_idx=500
    )
    #Save results
    file_name = f"{model}_results_null_pair_{comparison_models[0]}_{comparison_models[1]}.json"
    print(file_name)
    path = os.path.join("results","cnn","null_pair",file_name)
    save_to_json(results,path)

Starting gpt4
['gpt4', 'gpt35']


Processing keys: 100%|██████████| 500/500 [04:20<00:00,  1.92it/s]

gpt4_results_null_pair_gpt4_gpt35.json


## When Neither Pairs are yours but you have to pick one (False Pair)

In [18]:
# Check how good GPT is at recognizing first model as its own (both wrong)
def generate_gpt_detect_recognition_false_pair(
    dataset,
    model,
    comparison_models,
    starting_idx=0
):

    # For retrieving summaries, the specific fine-tuning version isn't needed
    exact_model = model
    model = "gpt35" if model.endswith("gpt35") else model

    responses, articles, keys = load_data(dataset)
    results = []  
    switch = False # shuffle between the two comparison models

    for key in tqdm(keys[starting_idx:], desc="Processing keys"):
        article = articles[key]
        first_summary = responses[comparison_models[0]][key]
        second_summary = responses[comparison_models[1]][key]
        
        if switch:
             first_summary, second_summary = second_summary, first_summary

        switch = not switch # flip every iteration, first iteration is False

        result = {"key": key}
        forward_result = get_model_choice(
            first_summary,
            second_summary,
            article,
            "detection",
            exact_model,
            return_logprobs=True,
        )
        if not switch:
            logprob = next((item.logprob for item in forward_result if item.token == '1'), None) 
        else:
            logprob = next((item.logprob for item in forward_result if item.token == '2'), None) 
        
        result["detection_score"] = exp(logprob)
        results.append(result)
    return results


In [19]:
for model in ["gpt4"]:
    
    print(f"Starting {model}")
    comparison_models =  ["gpt35", "human"] ## Change for the two models
    print(comparison_models)
    results = generate_gpt_detect_recognition_false_pair(
        "cnn", model,comparison_models,  starting_idx=500
    )
    #Save results
    file_name = f"{model}_results_false_pair_{comparison_models[0]}_{comparison_models[1]}.json"
    print(file_name)
    path = os.path.join("results","cnn","null_pair",file_name)
    save_to_json(results,path)

Starting gpt4
['gpt35', 'human']


Processing keys: 100%|██████████| 500/500 [05:04<00:00,  1.64it/s]

gpt4_results_false_pair_gpt35_human.json
